# Working with immune repertoire sequence data using `R`

We will be using `tidyverse` and `Bioconductor` to work with pre-processed immune repertoire sequence data in `R`.
You have already learned about both of these collections of packages, and today's class will give you the opportunity to practice using them in a new biological context.

Goals for today's class:
1. Familiarize with the format of immune repertoire sequence data
2. Work through an example analysis, and practice on your own

## Load packages

In [1]:
library(Biostrings)
library(tidyverse)

options(repr.plot.width = 7, repr.plot.height = 4)

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: S4Vectors

Loading required package: stats4


Attaching package: ‘S4Vectors’


The following objects are masked from ‘package:base’:

    expand.grid, I, unname


Loading required package: IRanges

Loading required package: XVector

Loading required package: GenomeInfoDb


Attaching package: ‘Biostrings’


The following object is masked from ‘package:base’:



## Loading the data

In [4]:
indiv1 <- read.csv('https://drive.google.com/uc?id=1b1oXlhg99_YCPK_HzFL2ohYPDxvTbQOo', sep = '\t', header = TRUE)
indiv2 <- read.csv('https://drive.google.com/uc?id=10qMY16H9wD_wuC4ISAJRTRx_bOeclg5D', sep = '\t', header = TRUE)

Let's take a look at the data. 
Recall that each annotated sequence takes the form:

5'-[__Vgene__]-(_possible Vgene deletion_)-[__N1insertion__]-(_possible Dgene deletion_)-[__D gene__]-(_possible Dgene deletion_)-[__N2insertion__]-(_possible Jgene deletion_)-[__J gene with possible deletion__]-3'

Using the column names given by the file, we can interpret this as:

5'-[__`v_gene`__]-(_`v_trim`_)-[__`vd_insert`__]-(_`d0_trim`_)-[__`d_gene`__]-(_`d1_trim`_)-[__`dj_insert`__]-(_`j_trim`_)-[__`j_gene`__]-3'

In [ ]:
head(indiv2)

## Some example analyses using `tidyverse` and `Bioconductor`

Find which V-gene occurs most frequently for individual 2

In [ ]:
Vcounts2 <- indiv2 %>%
                group_by(v_gene) %>%
                summarise(total_count = n()) %>%
                arrange(desc(total_count))

head(Vcounts2)

Plot the distribution of trimming lengths for that gene

In [ ]:
# isolate most frequently used V-gene
most_frequent_V <- Vcounts2 %>%
                    filter(total_count == max(Vcounts2$total_count))

head(most_frequent_V)

In [ ]:
# filter original data set by most frequently used V-gene
most_frequent_V_data <- indiv2 %>%
                            filter(v_gene == most_frequent_V$v_gene)
head(most_frequent_V_data)

In [ ]:
# plot trimming distribution
most_frequent_V_data %>%
    ggplot(aes(x = v_trim)) +
    geom_density()

Calculate V-D N-insertion composition frequencies within individual 2

In [ ]:
# filter for sequences that have VD N-inserts
n_indiv2 <- indiv2 %>%
                filter(vd_insert != 0)

In [ ]:
# convert VD N-insert column to a BioStrings `DNAStringSet`
nucs_indiv2 <- DNAStringSet(n_indiv2$vd_insert_nucs)

nucs_indiv2

In [ ]:
# get frequencies
nucs_indiv2 %>%
  letterFrequency(c("A", "T", "C", "G"), collapse = TRUE, as.prob = TRUE)

# In-class exercises

(20-30 minutes)

In [ ]:
# first let's combine the data into a single dataframe
indiv1$id = 1
indiv2$id = 2

together = rbind(indiv1, indiv2)
together

### 1. Plot the distributions of V-gene trimming for each V-gene (using a single plot). Find the V-gene with the largest average number of nucleotides trimmed.

In [ ]:
# plot distributions of trimming for each V-gene in a single plot (one line per gene)
most_trimmed_v = together %>%
    group_by(v_gene) %>%
    mutate(avg_trim = mean(v_trim)) %>%
    ungroup() %>%
    filter(avg_trim == max(avg_trim))
print(unique(most_trimmed_v$v_gene))
print(unique(most_trimmed_v$avg_trim))

In [ ]:
#colors = together$v_gene
together_plot = together %>%
    ggplot(aes(x=v_trim, group = v_gene)) + 
    geom_density()

together_plot

In [ ]:
# find the V-gene with the largest average number of trims
v_avg = mean(sum(together$v_trim))
v_avg

d_avg = mean(sum(together$d5_trim+together$d3_trim))
d_avg

j_avg = mean(sum(together$j_trim))
j_avg

### 2. Find the N-insertion base frequencies for each N-insertion junction (combining data from both individuals). Are they similar?

In [ ]:
# first find insertion frequencies for VD junction

vd_nuc = DNAStringSet(together$vd_insert_nucs)

vd_nuc %>% letterFrequency(c("A", "T", "C", "G"), collapse= TRUE, as.prob=TRUE)

In [ ]:
# then find insertion frequencies for DJ junction
dj_nuc = DNAStringSet(together$dj_insert_nucs)

dj_nuc %>% letterFrequency(c("A", "T", "C", "G"), collapse= TRUE, as.prob=TRUE)

### 3. Find the rearrangement/s (`cdr3` column) which have the largest overlap between the two individuals. Pick one of the largest overlap rearrangements--which nucleotide rearrangment most commonly leads to that CDR3 amino acid sequence for each individual?

_Note: there are many ways to do this..._

_Hint: you will want to work with the dataframes `indiv1` and `indiv2` instead of the full dataset_

_Hint 2: you will want to filter each dataframe for rows where the cdr3 column is not empty (ie. `cdr3 != ''`)_

_Hint 3: the `pmin` command will help you calculate the overlapping number shared between two columns (e.g. if individual 1 has 2 occurances of X cdr3 and individual 2 has 8 occurances of X cdr3, then the overlap will be 2)_

In [ ]:
# find the cdr3 that has the largest overlap between the two individuals (there may be multiple...)

indiv1_filtered = indiv1 %>%
    filter(cdr3 != '')

indiv2_filtered = indiv2 %>%
    filter(cdr3 != '')

cdr3_indiv1 = 


overlap = cdr3_tog %>%
    mutate(overlap_count = pmin(cdr3_count_indiv1, cdr3_count_indiv2)) 

head(overlap)

In [ ]:
# choose a single largest overlap cdr3

In [ ]:
# now, for each individual, figure out which nucleotide rearrangement (cdr3_nucseq) most commonly leads to that CDR3 amino acid sequence